In [372]:
from PyPDF2 import PdfFileReader
import os
import json
import pandas as pd

Extract information from pdf file

In [373]:
def extract_pdf(path):
    pdf=PdfFileReader(path)
    number_pages=pdf.getNumPages()
    content=''
    for i in range(number_pages):
        content+=pdf.getPage(i).extractText()
    
    content=content.replace('\\n','. ').replace('\n\n','. ')
    content=content.replace('\n','. ')
    content=content.replace('\t',' ')
    
    return content

Get path to each pdf file stored in folder  

In [374]:

def get_path_pdf(path_folder):
    path_pdf=[]
    for dirname,_,filenames in os.walk(path_folder):
        for filename in filenames:
            path_pdf.append(os.path.join(dirname,filename))
    return path_pdf


Read JD

In [375]:
path_JD="./JD/"
path_2_pdfJD=get_path_pdf(path_JD)
content_JD=extract_pdf(path_2_pdfJD[0])


Read CV

In [376]:
path_CV="./CV/"
path_2_pdfCV=get_path_pdf(path_CV)
content_CV=[]
for p in path_2_pdfCV:
    content_CV.append(extract_pdf(p))


Using Spacy to extract skill

In [377]:
from spacy.pipeline import EntityRuler
from spacy import displacy
import jsonlines
import spacy
import re 
import pyperclip
from sympy import true

In [378]:
json_path="skill_paterns.jsonl"

with jsonlines.open(json_path) as f:
    entities=[line["label"].upper() for line in f.iter()]

# spacy model
nlp=spacy.load("en_core_web_lg")

In [379]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(json_path)

PERSONAL INFORMATION

In [380]:

def get_skill(content_CV):
    doc=nlp(content_CV)
    skill_list=[]
    for ent in doc.ents:
        if ent.label_=='SKILL':
            skill_list.append(ent.text)
    skill_list=list(set(skill_list))
    return skill_list

def get_GPA(content_CV):
    doc=nlp(content_CV)
    sent=list(doc.sents)
    gpa=''
    for s in sent:
        s=s.text
        if s.find('GPA') !=-1 :
            gba_pos= s.find('GPA') 
            for i in range (gba_pos+3,gba_pos+8):
                if (s[i] >'0' and s[i]<'9' or s[i]=='.'):
                    gpa+=s[i]
    if gpa=='':
        return None
    else:
        return gpa

def get_experience(content_CV):
    '''
    Return\n
        None if no experience\n
        List of experience if exist
    '''
    doc=nlp(content_CV)
    sent=list(doc.sents)
    final_res=''
    ex=[]
    for s in sent:
        s_text=s.text.split(' ')
        flag=False
        for i in s_text:
            i=i.lower()
            if i=='experience' or i=='project':
                flag=True
        if flag==True:
            ex.append([s.text])
    if (len(ex)==0):
        final_res=None
    else:
        final_res=ex
    return final_res

def get_education(content_CV):
    doc=nlp(content_CV)
    sent=list(doc.sents)
    edu=''
    pos_edu=-1
    pos_edu_end=-1
    for s in sent:
        s_text=s.text.lower()
        flag=False
        if s_text.find('education')!=-1 or s_text.find('university')!=-1 or s_text.find('academic')!=-1:
            edu =s.text
            if s_text.find('education')!=-1:
                pos_edu=s_text.find('education')+9
            if s_text.find('academic') !=-1 :
                pos_edu=s_text.find('academic')
            else:
                pos_edu=s_text.find('university')
            
            pos_edu_end=s_text.find('. ',pos_edu)
            edu=edu[pos_edu:pos_edu_end]
    if edu=='':
        return None
    else:
        return edu

def get_phone(content_CV):
    '''
    Format for Phone-Numbers:
    134567890
    123-456-7890
    (123)(456)(7890)
    123.456.7890
    (123)(456-7890)
    '''
    phone= re.compile(r'''
        (\d{3}|\(\d{3}\))    
        (\s|-|\.)?           
        (\d{3}|\(\d{3}\))    
        (\s|-|\.)?           
        (\d{4}|\(\d{4}\))    
    ''', re.VERBOSE) 
    matchs=phone.finditer(content_CV)
    res = ''
    for match in matchs:
        res+=match.group(0)
    if res=='':
        return None
    else:
        return res

def get_link(link,content_CV):
    matchs=link.finditer(content_CV)
    res=''
    for match in matchs:
        res+=match.group(0)
    if res=='':
        return None
    else:
        return res

test=content_CV[0]
print('SKILL    :',get_skill(test))
print('GPA      :',get_GPA(test))       
print('Ex       :',get_experience(test))
print()
print('EDU      :',get_education(test))
print('Personal Website :',get_link(web,test))
print('GIT      :',get_link(git,test))
print('Linkedin :',get_link(linkedin,test))
print('Mail     :',get_link(mail,test))

print('Phone    :',get_phone(test))
test

SKILL    : ['CSS', 'Software', 'API', 'Javascript', 'languages', 'Programming Language', 'HTML', 'LANGUAGES', 'software', 'VueJS']
GPA      : None
Ex       : None

EDU      : ACADEMIC HISTORY
Personal Website : None
GIT      : None
Linkedin : None
Mail     : phanhoangphuc123321@gmail.com
Phone    : None


'MAY 2018 - PRESENT. Information Technology.THU DUC TECHNOLOGY COLLEGEACADEMIC HISTORY. - Programming Language: HTML, CSS, Javascript,  .   Ant Design Vue.  . - Knowledge: UX/UI, ReactJS, VueJS, .   GraphQL, RESTful API.     . - Teamworking.. - Highly responsible.. - Well-organized and disciplined.SKILLS. - Reading.. - Learning foreign languages.. - Playing soccer and gym.INTERESTS. - Vietnamese.. - English.LANGUAGES. - Phone: +84 786 572 491.. - Email: phanhoangphuc123321@gmail.com.. - Address: Quarter 6, Linh Trung Ward, Thu Duc. -. - District, Ho Chi Minh City, Vietnam.CONTACT INFO. - In my mind, learning through working is the most. important factor in my career path. That is the. reason why I’m looking for a opportunity at web. field company, to dedicate, and also. improve myself.ABOUT ME. ICT24H CO., LTD / Jun 2020 - Present. - Online Exam Software.. Project: Create Online Exam Software.. Description: This software could help. administrators creating exams quickly, save the. test

NAME

In [382]:
from spacy.matcher import Matcher
# initialize matcher with a vocab

def extract_name(content_CV):
    matcher = Matcher(nlp.vocab)
    doc=nlp(content_CV)
    for ent in doc.ents:
        if ent.label_=='PERSON':
            print(ent.label_,'  ',ent.text)
            return(ent.text)
            
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'},{'POS': 'PROPN'}]
    matcher.add('NAME',[pattern])
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        return span.text
        
    return None

extract_name(content_CV[0])

PERSON    Linh Trung Ward


'Linh Trung Ward'

Check Accuracy of name

In [ ]:
train_data = json.load(open('./data/training/train_data.json','rb')) 

def name_acc(train_data):
    list_an=[]
    list_txt=[]
    name=[]

    for text,annotation in train_data:
        list_an.append(list(annotation.values()))
        list_txt.append(text)

    target_name=[]
    predic_name=[]

    similar=0
    for i in range(len(list_an)):
        t=''
        p=''
        p=extract_name(list_txt[i])
        predic_name.append(p)
        for j in range(len(list_an[i][0])):
            if list_an[i][0][j][2]=='Name':
                t=list_txt[i][list_an[i][0][j][0]:list_an[i][0][j][1]]
                target_name.append(t)
        if t==p:
            similar+=1
    return (similar/len(list_an)*100)    
name_acc(train_data)

Final extract and save to dataframe

In [390]:
label=['ID','name','email','phone number','related link(git,web,linkedin)','skill','experience','education','GPA']
df_information = pd.DataFrame(columns=label) 
df_information

,ID,name,email,phone number,"related link(git,web,linkedin)",skill,experience,education


In [397]:
def extract_all_data(list_content_CV, df):
    mail = re.compile(r'[a-zA-Z0-9-\.]+@[a-zA-Z-\.]*\.(com|edu|net)')
    git = re.compile(r'(gitlab.com/|github.com/)+[a-zA-Z0-9-\.]*')
    web = re.compile(r'(http://|https:// )+[a-zA-Z0-9-"/"-\.]*')
    linkedin = re.compile(r'linkedin.com/+[a-zA-Z0-9-"/"-\.]*')

    for i in range(len(list_content_CV)):
        content=list_content_CV[i]
        link=[]
        link.append(get_link(git,content))
        link.append(get_link(web,content))
        link.append(get_link(linkedin,content))
        r = {
            'ID': i,
            'name': extract_name(content),
            'email': get_link(mail,content),
            'phone number': get_phone(content),
            'related link(git,web,linkedin)': link,
            'skill': get_skill(content),
            'experience': get_experience(content),
            'education': get_education(content)
            'GPA':get_GPA(content)
            }

        df = df.append(r, ignore_index=True)
    return df


extract_all_data(content_CV,df_information)
# test=content_CV[4]
# print(get_GPA(test))       
# print(get_experience(test))
# print()
# print(get_education(test))
# print(get_link(web,test))
# print(get_link(git,test))
# print(get_link(linkedin,test))
# print(get_link(mail,test))
# print(get_skill(test))
# get_phone(test)

PERSON    Linh Trung Ward


C:\Users\Admin\AppData\Local\Temp\ipykernel_8824\2885845269.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r, ignore_index=True)


PERSON    Chi Minh City


C:\Users\Admin\AppData\Local\Temp\ipykernel_8824\2885845269.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r, ignore_index=True)


PERSON    Volunteer


C:\Users\Admin\AppData\Local\Temp\ipykernel_8824\2885845269.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r, ignore_index=True)


PERSON    Phan Thi Bich Ngoc Curriculum Vitae


C:\Users\Admin\AppData\Local\Temp\ipykernel_8824\2885845269.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r, ignore_index=True)


,ID,name,email,phone number,"related link(git,web,linkedin)",skill,experience,education
0,0,Linh Trung Ward,phanhoangphuc123321@gmail.com,None,"[None, None, None]","[CSS, Software, API, Javascript, languages, Pr...",None,"[None, ACADEMIC HISTORY]"
1,1,Chi Minh City,nguyenkhavi01032001@gmail.com,0374246292,"[gitlab.com/nguyenkhavi., None, linkedin.com/i...","[natural language processing, API, mobile, NLP...",[[A software developer with 1+ year of experie...,"[8.3, University of Science]"
2,2,Volunteer,levietdc01@gmail.com,0396024273,"[None, None, None]","[Visual Studio Code, Hadoop, Computer Vision, ...",None,"[81, University of Science, majoring in Comput..."
3,3,Phan Thi Bich Ngoc Curriculum Vitae,Ptbichngoc2502@gmail.com,0347297090,"[None, None, linkedin.com/in/bich-ngoc-0ab1831...","[Computer vision, database, OpenGL, SQL, CERTI...",[[guide on the various steps a project goes th...,"[7.5, University of Science]"


Load accuracy name

In [ ]:
nlp_model=spacy.load('nlp_model')

Using sklearn to calculate the cosine similarity score of CV and JD

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# def cosine_similarity(list_CV,JD):
#     test=[list_CV]
test=[content_CV,content_JD]
count_matrix=cv.fit_transform(test)
cosine_similarity(count_matrix)

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
test=[content_CV[4],content_JD]
count_matrix=cv.fit_transform(test)
match=cosine_similarity(count_matrix)
match[0][1]

0.4040562886306343

In [ ]:
count_matrix

<2x244 sparse matrix of type '<class 'numpy.int64'>'
	with 244 stored elements in Compressed Sparse Row format>